**TO DO:** Make a copy of this notebook in your own Google drive and edit the copy.

**TO DO:** Download the data at the following link https://stanfordmlgroup.github.io/competitions/mrnet/ and upload it to your Google Drive. 

This will download a folder named 'data'. 

*   The dataset consists of 1,250 knee MRIs with image level labels.
*   The training data consists of 1,130 MRIs and the validation data consists of 120 MRIs.
*   They are labelled as abnormal, having an acl tear and/or meniscus tear.
*   Each MRI exam includes data from the axial, coronal and sagittal plane. 
*   Axial is a Proton-Density series, coronal is a T1-weighted series and sagittal is T2-weighted series.






Go to "Edit" on the toolbar, then "Notebook Settings" and change the hardware accelerator to GPU.


#Mount Google Drive to access your data



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


The following code uses a python library named 'torchsample'. This is not installed in Google Colab. We can import it by running the commands in the following cell. The exclamation mark communicates to Google Colab to run the commands in the terminal rather than in Python in the current notebook.


In [ ]:
!pip install -e git+https://github.com/ncullen93/torchsample.git#egg=torchsample
!pip install visdom
!pip install nibabel
!pip install h5py  
!pip install torchsample
!pip install tensorboardX

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining torchsample from git+https://github.com/ncullen93/torchsample.git#egg=torchsample
  Updating ./src/torchsample clone
  Running command git fetch -q --tags
  Running command git reset --hard -q 1f328d1ea3ef533c8c0c4097ed4a3fa16d784ba4
  Attempting uninstall: torchsample
    Found existing installation: torchsample 0.1.3
    Can't uninstall 'torchsample'. No files were found to uninstall.
  Running setup.py develop for torchsample
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python

In [ ]:
#import all libraries
import torch.optim as optim
import torch
import torch.nn as nn
from torchvision import models
import numpy as np
import os
import sys
import pickle
import torch.nn.functional as F
import torch.utils.data as data
import pandas as pd
from torch.autograd import Variable
from src.torchsample.torchsample.transforms import RandomRotate, RandomTranslate, RandomFlip, ToTensor, Compose, RandomAffine
from torchvision import transforms
from tensorboardX import SummaryWriter
import math
from sklearn import metrics

#Define your model
The model is defined in the class 'Net'. The 'init' function initialises the architecture of the model.

The line of code; ```self.pretrained_model = None``` should be edited to initialise a pre-trained model, pre-trained on the ImageNet Dataset. This initialises the weights of the model with the weights for a pre-trained model that was trained on the ImageNet dataset. This speeds up training.

The line of code ```self.classifer = None``` should be edited to be a fully connected layer that makes the final prediction.

After the model is initialised, the forward function is called iteratively throughout the training process. 

More information con defining models can be found at https://pytorch.org/vision/stable/models.html

##Fill in None values

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.pretrained_model = models.resnet18(pretrained=True)
        self.conv = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=1)
        self.soft = nn.Softmax(2)
        self.classifer = nn.Linear(1000, 1)

    def tile(a, dim, n_tile):
        init_dim = a.size(dim)
        repeat_idx = [1] * a.dim()
        repeat_idx[dim] = n_tile
        a = a.repeat(*(repeat_idx))
        order_index = torch.LongTensor(np.concatenate([init_dim * np.arange(n_tile) + i for i in range(init_dim)]))
        if torch.cuda.is_available():
            a = a.cuda()
            order_index = order_index.cuda()
        return torch.index_select(a, dim, order_index)

    def forward(self, x):
        x = torch.squeeze(x, dim=0)
        x = self.pretrained_model.conv1(x)
        x = self.pretrained_model.bn1(x)
        x = self.pretrained_model.maxpool(x)
        x = self.pretrained_model.layer1(x)
        x = self.pretrained_model.layer2(x)
        x = self.pretrained_model.layer3(x)
        x = self.pretrained_model.layer4(x)
        attention = self.conv(x)
        attention =  self.soft(attention.view(*attention.size()[:2], -1)).view_as(attention)
        maximum = torch.max(attention.flatten(2), 2).values
        maximum = Net.tile(maximum, 1, attention.shape[2]*attention.shape[3])
        attention_norm = attention.flatten(2).flatten(1) / maximum
        attention_norm= torch.reshape(attention_norm, (attention.shape[0],attention.shape[1],attention.shape[2],attention.shape[3]))
        o = x*attention_norm
        out= self.pretrained_model.avgpool(o)
        out = self.pretrained_model.fc(out.squeeze())
        output = torch.max(out, 0, keepdim=True)[0]
        output = self.classifer(output)

        return output


#Create Dataloader
The 'init' function initialises the dataloader. This class is responsible for loading the datasets. It takes the 'root_dir', 'task', 'plane', 'train' and 'transform' as input parameters. 
root_dir - the directory to where the data is stored.

task - whether the model is being trained to detect acl tears, meniscus tears or abnormalities. Possible values are 'acl', 'meniscus' or 'abnormal'.

plane - whether the model is being trained on axial, coronal or sagittal data. Possible values are 'axial', 'coronal' or 'sagittal'.

train - is this the dataloader for the training data or the validation data. Possible values are 'True' to load training data or 'False' to load validation data.

transform - a compose function for performing transformations to the images.

The init function creates 1) a list of paths to each MRI, 2) a corresponding list of labels that are either ones or zeros and 3) weights.


---



The __len__ function returns the length of the dataset.


---
The __getitem__ function is iteratively called throughout the training process. It takes an index as a input parameter. It loads the MRI at the given index from the list of paths defined in the init function. It also returns the label and weight for the MRI at that index.



In [ ]:
class Dataset(data.Dataset):
    def __init__(self, root_dir, task, plane, train=False, transform=None):
        super().__init__()
        self.task = task
        self.plane = plane
        self.root_dir = root_dir
        self.train=train
        if self.train == True:
            self.folder_path = self.root_dir + 'train/{0}/'.format(plane)
            self.records = pd.read_csv(
                self.root_dir + 'train-{0}.csv'.format(task), header=None, names=['id', 'label'])
        else:
            self.folder_path = self.root_dir + 'valid/{0}/'.format(plane)

            self.records = pd.read_csv(
                self.root_dir + 'valid-{0}.csv'.format(task), header=None, names=['id', 'label'])

        self.records['id'] = self.records['id'].map(
            lambda i: '0' * (4 - len(str(i))) + str(i))
        self.paths = [self.folder_path + filename +
                      '.npy' for filename in self.records['id'].tolist()]
        self.labels = self.records['label'].tolist()

        self.transform = transform
        
        pos = np.sum(self.labels)
        neg = len(self.labels) - pos
        self.weights = [1, neg / pos]
        

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, index):
        array = np.load(self.paths[index]) #load MRI 
        label = self.labels[index] #get label of MRI
        label = torch.FloatTensor([label]) #convert type from numpy to torch

        if self.transform: #if you are transforming it
            array = self.transform(array) #transform the image
            array = array.numpy()


        array = np.stack((array,)*3, axis=1) #the model expects dimensions of (3, 256, 256), the MRIs are greyscale of size (256, 256). Therefore, we stack the image three times to fit the dimensions for the model.
        array = torch.FloatTensor(array)

        if label.item() == 1:
            weight = np.array([self.weights[1]])
            weight = torch.FloatTensor(weight)
        else:
            weight = np.array([self.weights[0]])
            weight = torch.FloatTensor(weight)

        return array, label, weight

#Train the model
##Define variables
**TO DO:** Change directory to where you store your data. Use the toolbar to the side of this page to view your file system.

In [ ]:
directory='./gdrive/MyDrive/MRNet-v1.0/' #directory to the data
task = 'acl'
plane = 'sagittal'
lr = 1e-5 #learning rate
num_epochs = 5 # number of epochs

##Initialise the model, optimiser, scheduler, transformations and data loader.

##Fill in the 'None' values

In [ ]:
model = Net() #initialise the model
if torch.cuda.is_available(): #if there is a GPU available, put the model on the GPU
    model = model.cuda()

optimizer = optim.Adam(model.parameters(), lr= lr, weight_decay=0.1) #define the optimiser as Adam

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, patience=4, factor=.3, threshold=1e-4, verbose=True) #define a scheduler that decreases the learning rate if there has been no reduction in validation loss is four epochs

#define a compose function that is a series of transformations on the images. 
augmentor = Compose([ 
            transforms.Lambda(lambda x: torch.Tensor(x)), #converts from numpy to tensor
            RandomRotate(25), #rotate the image by 25 degrees
            RandomTranslate([0.11, 0.11]), #blur the edges
            RandomFlip(), #flip the image
        ])

#initialise the train and validation datasets (class we defined earlier) and then initialise a Pytorch's dataloader
train_dataset = Dataset(directory, task, plane, train=True, transform=augmentor) #create a Dataset object and pass the augmentor the class
valid_dataset = Dataset(directory, task, plane, train=False, transform = None) #create a Dataset object for the validation data, don't transform the validation data

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=1, shuffle=True, num_workers=2, drop_last=False)
valid_loader = torch.utils.data.DataLoader(
    valid_dataset, batch_size=1, shuffle=-True, num_workers=2, drop_last=False)

##Training Loop

##Fill in the 'None' values

In [ ]:
early_trigger = 10 #if the validation AUC hasn't increased in ten epochs, stop the training
early_stop = 0 #counter for the number of iterations where there has been no increase in validation AUC
best_val_auc = 0
counter = 0
progress = []
#for loop for each epoch
for epoch in range(num_epochs):
      #get learning rate
      current_lr = lr
      _ = model.train()
      y_preds = []
      y_trues = []
      losses = []
      

      #loop through each MRI in the training set
      for i, (image, label, weight) in enumerate(train_loader):
          optimizer.zero_grad()

          #load all data onto the GPU
          if torch.cuda.is_available():
              image = image.cuda()
              label = label.cuda()
              weight = weight.cuda()

          label = label[0]
          weight = weight[0]

          #pass the MRI through the model
          prediction = model.forward(image.float()).squeeze(0)

          #calculate the loss
          loss = torch.nn.BCEWithLogitsLoss(weight=weight)(prediction, label)
          loss.backward() #back propagation
          optimizer.step()

          counter += 1
          if (counter % 10 == 0):
            progress.append(loss.item())
            pass

          loss_value = loss.item()
          losses.append(loss_value)

          probas = torch.sigmoid(prediction) #convert output of model (logits) to a value between zero and one using torch.sigmoid(). This can be interpretted as a probability

          y_trues.append(int(label[0]))
          y_preds.append(probas[0].item())

          try:
              auc = metrics.roc_auc_score(y_trues, y_preds)
          except:
              auc = 0.5

          train_loss = np.round(np.mean(losses), 4)
          train_auc = np.round(auc, 4)

      #evaluate the model on the validation data after each epoch
      _ = model.eval()
      y_trues = []
      y_preds = []
      losses = []
      for i, (image, label, weight) in enumerate(valid_loader):

        if torch.cuda.is_available():
            image = image.cuda()
            label = label.cuda()
            weight = weight.cuda()

        label = label[0]
        weight = weight[0]

        prediction = model.forward(image.float()).squeeze(0)

        loss = torch.nn.BCEWithLogitsLoss(weight=weight)(prediction, label)

        loss_value = loss.item()
        losses.append(loss_value)

        probas = torch.sigmoid(prediction)

        y_trues.append(int(label[0]))
        y_preds.append(probas[0].item())

        try:
            auc = metrics.roc_auc_score(y_trues, y_preds)
        except:
            auc = 0.5

        val_loss = np.round(np.mean(losses), 4)
        val_auc = np.round(auc, 4)

      if val_auc > best_val_auc:
        best_val_auc = val_auc
        early_stop=0
      else:
        early_stop+= 1

      if early_stop == early_trigger:
        print('Early stopping after {} epochs'.format(epoch))
        sys.exit()
      scheduler.step(val_loss)

      print("epoch : {0} | train loss : {1} | train auc {2} | val loss {3} | val auc {4} ".format(
          epoch, train_loss, train_auc, val_loss, val_auc))

      
      print('-' * 30)

epoch : 0 | train loss : 1.133 | train auc 0.5356 | val loss 0.7261 | val auc 0.7264 
------------------------------
epoch : 1 | train loss : 1.1034 | train auc 0.605 | val loss 0.7098 | val auc 0.7904 
------------------------------
epoch : 2 | train loss : 1.0747 | train auc 0.67 | val loss 0.6943 | val auc 0.8302 
------------------------------
epoch : 3 | train loss : 1.0141 | train auc 0.7563 | val loss 0.6489 | val auc 0.828 
------------------------------
epoch : 4 | train loss : 0.9627 | train auc 0.7801 | val loss 0.6174 | val auc 0.8336 
------------------------------


In [ ]:
import pandas

def plot_progress():
    df = pandas.DataFrame(progress, columns=['loss'])
    df.plot(ylim=(0, 1.5), figsize=(16,8), alpha=0.1, marker='.', grid=True, yticks=(0, 0.25, 0.5, 0.75, 1, 1.25))
    pass

In [ ]:
plot_progress()

NameError: ignored

In [ ]:
from sklearn.metrics import RocCurveDisplay
RocCurveDisplay.from_predictions(y_trues, y_preds)